In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("="*70)
print("STEP 1: LOADING RAW DATA")
print("="*70)

# Load the datasets
marketing_spend = pd.read_csv('../data/marketing_spend.csv', parse_dates=['Date'])
sales = pd.read_csv('../data/sales.csv', parse_dates=['Date'])
brand_metrics = pd.read_csv('../data/brand_metrics.csv', parse_dates=['Date'])
competitor_activity = pd.read_csv('../data/competitor_activity.csv', parse_dates=['Date'])
macroeconomic_indicators = pd.read_csv('../data/macroeconomic_indicators.csv', parse_dates=['Date'])

print(f"✓ Loaded marketing_spend: {len(marketing_spend)} rows")
print(f"✓ Loaded sales: {len(sales)} rows")
print(f"✓ Loaded brand_metrics: {len(brand_metrics)} rows")
print(f"✓ Loaded competitor_activity: {len(competitor_activity)} rows")
print(f"✓ Loaded macroeconomic_indicators: {len(macroeconomic_indicators)} rows")

print(f"\nMarketing channels in data: {marketing_spend['Channel'].unique().tolist()}")

# Phase 1: Data Preparation & Exploratory Analysis

This notebook prepares the Marketing Mix Modeling (MMM) data by:
1. Loading raw data files
2. Merging datasets
3. Handling missing values
4. Validating data quality
5. Creating the final prepared dataset for modeling

**Output:** `prepared_data.csv` - ready for UCM-MMM and BVAR models

## 1. Data Cleaning & Transformation

In [ ]:
print("\n" + "="*70)
print("STEP 2: MERGING DATASETS")
print("="*70)

# Pivot marketing spend data to have channels as columns
marketing_spend_pivot = marketing_spend.pivot_table(
    index='Date', 
    columns='Channel', 
    values='Spend'
).reset_index()

print(f"✓ Pivoted marketing spend to wide format")
print(f"  Columns: {marketing_spend_pivot.columns.tolist()}")

# Merge the datasets into a single dataframe
df = pd.merge(sales, marketing_spend_pivot, on='Date', how='left')
df = pd.merge(df, brand_metrics, on='Date', how='left')
df = pd.merge(df, competitor_activity, on='Date', how='left')
df = pd.merge(df, macroeconomic_indicators, on='Date', how='left')

print(f"\n✓ Merged all datasets")
print(f"  Final shape: {df.shape}")
print(f"  Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"  Total weeks: {len(df)}")

# Check for missing values before handling
print(f"\nMissing values before handling:")
missing = df.isnull().sum()
print(missing[missing > 0])

# Handle missing values
# Forward fill for brand and macro data (surveys/indicators less frequent)
df[['Awareness', 'Consideration']] = df[['Awareness', 'Consideration']].ffill()
df[['GDP_Growth', 'Unemployment_Rate', 'Consumer_Confidence']] = \
    df[['GDP_Growth', 'Unemployment_Rate', 'Consumer_Confidence']].ffill()

# Fill remaining NaNs with 0 (for spend - weeks with no campaign)
df.fillna(0, inplace=True)

print(f"\n✓ Handled missing values")
print(f"  Missing values after: {df.isnull().sum().sum()}")

# Display first few rows
print(f"\nFirst 5 rows:")
print(df.head())

In [ ]:
print("\n" + "="*70)
print("STEP 4: SAVE PREPARED DATA")
print("="*70)

# Save the prepared data
output_path = '../data/prepared_data.csv'
df.to_csv(output_path, index=False)

print(f"✓ Saved prepared data to: {output_path}")
print(f"  Shape: {df.shape}")
print(f"  Columns: {len(df.columns)}")
print(f"\n✅ DATA PREPARATION COMPLETE!")
print(f"\nNext step: Run 02_Short_Term_Model.ipynb")

In [ ]:
# Visualize Revenue and Marketing Spend Over Time

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Revenue over time
ax1 = axes[0]
ax1.plot(df['Date'], df['revenue'], '-', color='#2E86AB', linewidth=2)
ax1.set_xlabel('Date', fontsize=12, fontweight='bold')
ax1.set_ylabel('Revenue ($)', fontsize=12, fontweight='bold')
ax1.set_title('Revenue Over Time', fontsize=14, fontweight='bold', pad=15)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Add mean line
mean_revenue = df['revenue'].mean()
ax1.axhline(mean_revenue, color='red', linestyle='--', alpha=0.5, 
            label=f'Mean: ${mean_revenue:,.0f}')
ax1.legend()

# Plot 2: Marketing spend over time by channel
ax2 = axes[1]
colors = {'Content Marketing': '#E63946', 'Events': '#F1C453', 
          'Google Ads': '#06A77D', 'LinkedIn': '#2E86AB'}

for channel in marketing_channels:
    ax2.plot(df['Date'], df[channel], '-', label=channel, 
             color=colors.get(channel, 'gray'), linewidth=2, alpha=0.8)

ax2.set_xlabel('Date', fontsize=12, fontweight='bold')
ax2.set_ylabel('Marketing Spend ($)', fontsize=12, fontweight='bold')
ax2.set_title('Marketing Spend Over Time by Channel', fontsize=14, fontweight='bold', pad=15)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\nRevenue Statistics:")
print(f"  Mean: ${df['revenue'].mean():,.0f}")
print(f"  Median: ${df['revenue'].median():,.0f}")
print(f"  Std Dev: ${df['revenue'].std():,.0f}")

print(f"\nTotal Marketing Spend by Channel:")
for channel in marketing_channels:
    total = df[channel].sum()
    print(f"  {channel}: ${total:,.0f}")

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Data Quality Checks

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller

print("="*70)
print("DATA QUALITY CHECKS")
print("="*70)

# 1. Multicollinearity Check (VIF)
print("\n1. Variance Inflation Factor (VIF)")
print("   (VIF > 10 indicates high multicollinearity)")
print("-" * 70)

X = df[['Content Marketing', 'Events', 'Google Ads', 'LinkedIn', 
        'Competitor_A_Spend', 'Competitor_B_Spend']]

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data = vif_data.sort_values('VIF', ascending=False)

print(vif_data.to_string(index=False))

high_vif = vif_data[vif_data['VIF'] > 10]
if len(high_vif) > 0:
    print(f"\n⚠ {len(high_vif)} variables have VIF > 10 (high multicollinearity)")
    print("   This is expected in MMM - hierarchical models handle this!")
else:
    print("\n✓ No high multicollinearity detected")

# 2. Stationarity Test (ADF) for Revenue
print("\n2. Augmented Dickey-Fuller Test for Revenue")
print("   (Tests if revenue series is stationary)")
print("-" * 70)

result = adfuller(df['revenue'])
print(f"ADF Statistic: {result[0]:.4f}")
print(f"p-value: {result[1]:.4f}")

if result[1] < 0.05:
    print("✓ Revenue series is stationary (p < 0.05)")
else:
    print("⚠ Revenue series is non-stationary (p >= 0.05)")
    print("  This is OK - UCM-MMM handles trends and seasonality")

# 3. Data completeness
print("\n3. Data Completeness")
print("-" * 70)
print(f"Total observations: {len(df)}")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Weeks covered: {(df['Date'].max() - df['Date'].min()).days / 7:.0f}")
print("\n✅ Data quality checks complete!")

## 3. Rigorous Statistical Filtering

In [5]:
# Multicollinearity Check (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

X = df[["LinkedIn", "Google Ads", "Content Marketing", "Events", "Competitor_A_Spend", "Competitor_B_Spend"]]
print(calculate_vif(X))

# Stationarity Test (ADF)
from statsmodels.tsa.stattools import adfuller

def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])

print("ADF Test for Revenue:")
adf_test(df['revenue'])

            variables        VIF
0            LinkedIn  23.393475
1          Google Ads  42.031202
2   Content Marketing  23.578007
3              Events   1.116223
4  Competitor_A_Spend  23.911370
5  Competitor_B_Spend  36.598378
ADF Test for Revenue:
ADF Statistic: -0.772983
p-value: 0.826930
